# 06 Constructing a Multimodal Cancer Network

In this notebook we show how to put together the mode and link tables that we constructed in the previous two notebooks ([04 Creating Mode Tables](04 Creating Mode Tables.ipynb), and [05 Creating Link Tables](05 Creating Link Tables.ipynb)) in order to build a multimodal cancer network.

## Step 0: Setup

We start by importing the required package and creating the output directory if it does not yet exist.

In [1]:
import snap
import os
import datetime

from utils.network_utils import load_mode_to_graph, load_crossnet_to_graph

today = datetime.date.today()
datestring = "%s%s%s" % (today.year, today.month, today.day)

## Step 1: Create an Empty Multimodal Network

First, we create an empty multimodal network. We proceed by adding modes and link types in the network.

In [2]:
context = snap.TTableContext()
# Graph object to hold multimodal cancer network
Graph = snap.TMMNet.New()

Multimodal network in Mambo is represented by an object of type [``snap.TMMNet``](https://snap.stanford.edu/snappy/doc/reference/multimodal.html#tmmnet). 

Object ``snap.TMMNet`` consists of modes, which are objects of type ``snap.TModeNet``, and links between modes, which are objects of type ``snap.TCrossNet``.

See [Reference manual](https://snap.stanford.edu/snappy/doc/reference/multimodal.html) for details.

## Step 2: Specify and Load Network Modes

Add all modes to the multimodal network. 

Note that mode tables for the multimodal cancer network were constructed in notebook [04 Creating Mode Tables](04 Creating Mode Tables.ipynb).

In [3]:
mode_table_filenames = [
    "datasets/cancer_example/chemical/miner-chemical-%s.tsv" % datestring,
    "datasets/cancer_example/disease/miner-disease-%s.tsv" % datestring,
    "datasets/cancer_example/function/miner-function-%s.tsv" % datestring,
    "datasets/cancer_example/gene/miner-gene-%s.tsv" % datestring,
    "datasets/cancer_example/protein/miner-protein-%s.tsv" % datestring,
]
for mode_table in mode_table_filenames:
    splitName = mode_table.split('-')
    load_mode_to_graph(splitName[1], mode_table, Graph, context)

## Step 3: Specify and Load Link Types

Add all link types to the multimodal network. 

Note that link tables for the multimodal cancer network were constructed in notebook [05 Creating Link Tables](05 Creating Link Tables.ipynb).

#### 3.1) Load all link types excluding gene-gene interactions and protein-protein interactions

We load all of the link types, excluding gene-gene interactions and protein-protein interactions. This is because there are many different types of protein-protein and gene-gene interactions. Technically, these different types of interactions between genes or proteins are taken into account by splitting gene-gene and protein-protein interactions into several link types. We handle gene-gene interactions and protein-protein interactions in the following part.

In [4]:
link_table_filenames = [
    "datasets/cancer_example/chemical-chemical/miner-chemical-chemical-%s.tsv" % datestring,
    "datasets/cancer_example/chemical-protein/miner-chemical-protein-%s.tsv" % datestring,
    "datasets/cancer_example/disease-chemical/miner-disease-chemical-%s.tsv" % datestring,
    "datasets/cancer_example/disease-disease/miner-disease-disease-%s.tsv" % datestring,
    "datasets/cancer_example/disease-function/miner-disease-function-%s.tsv" % datestring,
    "datasets/cancer_example/disease-protein/miner-disease-protein-%s.tsv" % datestring,
    "datasets/cancer_example/function-function/miner-function-function-%s.tsv" % datestring,
    "datasets/cancer_example/gene-protein/miner-gene-protein-%s.tsv" % datestring,
    "datasets/cancer_example/protein-function/miner-protein-function-%s.tsv" % datestring,
]

for link_table in link_table_filenames:
    link_table_name = os.path.basename(link_table)
    splitName = link_table_name.split('-')
    crossnetName = splitName[1] + "-" + splitName[2] + "-" + 'id'
    srcModeName = splitName[1]
    dstModeName = splitName[2]
    load_crossnet_to_graph(context, crossnetName, srcModeName, dstModeName, link_table, Graph)

#### 3.2) Load gene-gene interaction link types

Within gene-gene interactions, there are 6 different link types corresponding to different relationships between genes: co-expression, co-localization, genetic interactions, pathway co-membership, physical interactions, and predicted interactions. We load each of these link types separately.

In [5]:
types = [
'coexpression', 
'colocalization',
'genetic_interactions',
'pathway',
'physical_interactions',
'predicted',
]

for typ in types:
    link_table = "datasets/cancer_example/gene-gene/%s_links/miner-gene-gene-%s.tsv" % (typ, datestring)
    link_table_name = os.path.basename(link_table)
    splitName = link_table_name.split('-')    
    crossnetName = splitName[1] + '-' + splitName[2] + "-" + 'id'
    srcModeName = splitName[1]
    dstModeName = splitName[2]
    load_crossnet_to_graph(context, crossnetName, srcModeName, dstModeName, link_table, Graph, prefix=typ)

#### 3.3) Load protein-protein interaction link types

Within protein-protein interactions, there are 7 different link types corresponding to different relationship between proteins: co-expression, co-occurence, database evidence, experimental support, gene fusion, neighborhood on the genome, and text-mining. 

Note that these link types indicate different types of biological association, like co-expression, whereas other link types indicate approaach through which the information was collected, like experimental evidence and text-mining.

In [6]:
types = [
'coexpression', 
'cooccurence',
'database',
'experimental',
'neighborhood',
'textmining',
]

for typ in types:
    link_table = "datasets/cancer_example/protein-protein/%s_links/miner-protein-protein-%s.tsv" % (typ, datestring)
    link_table_name = os.path.basename(link_table)
    splitName = link_table_name.split('-')    
    cossnetName = splitName[1] + '-' + splitName[2] + "-" + 'id'
    srcModeName = splitName[1]
    dstModeName = splitName[2]
    load_crossnet_to_graph(context, crossnetName, srcModeName, dstModeName, link_table, Graph, prefix=typ)

## Step 4: Save Multimodal Network to a Disk

After the network has been constructed, we save the network to disk. 

This way the network can be loaded back into memoery very quickly and can be used again for downstream aanalysis without reconstructing the network from scratch. This property is especially appealing when dealing with very large networks (e.g., [09 Giga-Scale Multimodal Biological Network](09 Giga-Scale Multimodal Biological Network.ipynb)).

In [7]:
output_dir = "output/cancer_example"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [8]:
graph_name = "cancer_example.graph"
outputPath = os.path.join(output_dir, graph_name)

print "Saved in: %s" % outputPath
FOut = snap.TFOut(outputPath)
Graph.Save(FOut)
FOut.Flush()

Saved in: output/cancer_example/cancer_example.graph
